# Introduction to Content Safety

## Setup for Azure Content Safety
Azure Content Safety APIs are used with API key. A secured way for developers to get these keys is from Azure Key Vault. This pattern eliminates the need to store the secrets in your code.

Reference - https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/contentsafety/azure-ai-contentsafety

#### Prerequisites: 
- pip install azure-ai-contentsafety
- Your AAD Credentials have 'secrets' read access to Azure Key Vault
- The following OS environment variables are set - 
  - The OPENAI_AUTH_TYPE variable is set to KeysFromAKVWithCLIAuth or KeysFromAKVWithMI
  - KEY_VAULT_URL is set to your Key Vault to read secrets from
- The following secrets should be stored in the above Azure Key Vault
  - 'content-safety-api-key' set to the Content Safety API key
  - 'content-safety-endpoint' set to the Content Safety Endpoint

## Setup for Azure Open AI (only if the OpenAI APIs are used here)
#### Before you start these notebooks, make sure you have the OS environment variables setup. Example provided in the env.bat file for Windows in the root folder. Default variable settings for getting the OpenAI keys from your Azure Key Vault are 
- PYTHONPATH=%cd%\AzureOpenAIHelperFunctions
- KEY_VAULT_URL=https://\<YOUR AZURE KEY VAULT NAME\>.vault.azure.net/
- OPENAI_AUTH_TYPE=KeysFromAKVWithCLIAuth
#### The different options for OPENAI_AUTH_TYPE are - 
- KeysFromEnv => get the credentials from OS env
- KeysFromAKVWithCLIAuth => get the credentials from Azure Key Vault with CLI Auth
- KeysFromAKVWithMI => get the credentials from Azure Key Vault with Managed Identity
- KeysFromManagedId => authenticate with Managed Identity and get access to Azure Open AI
#### If using the Azure Key Vault to get the Azure Open AI  settings, set these secrets in the vault to the vaules you get from your Azure Open AI instance.
  - 'openai-api-key' set to the Azure Open AI API key
  - 'openai-api-version' set to the Azure Open AI API version
  - 'openai-endpoint' set to the Azure Open AI endpoint
  
#### Load the API key and relevant Python libaries.

In [2]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.contentsafety import ContentSafetyClient

from azure_content_safety_setup import set_content_safety_config

key, endpoint = set_content_safety_config()
credential = AzureKeyCredential(key)
client = ContentSafetyClient(endpoint, credential)

Getting Azure Content Safety Credentials from Azure Key Vault with Azure CLI Auth


#### Analyze text without blocklists

In [3]:
from azure.core.exceptions import HttpResponseError
from azure.ai.contentsafety.models import AnalyzeTextOptions, TextCategory

request = AnalyzeTextOptions(text="I will kil'ya all, you f**king mo**ns.", 
                             categories=[TextCategory.HATE, 
                                         TextCategory.SELF_HARM, 
                                         TextCategory.VIOLENCE,
                                         TextCategory.SEXUAL]
                            )
try:
    response = client.analyze_text(request)
except HttpResponseError as e:
    print("Analyze text failed.")
    if e.error:
        print(f"Error code: {e.error.code}")
        print(f"Error message: {e.error.message}")
        raise
    print(e)
    raise

text = 'Content Safety Results:'
# Check the severity (0 to 6)
if response.hate_result:
    text = text + "\n\tHate severity: {}".format(response.hate_result.severity)
if response.self_harm_result:
    text = text + "\n\tSelfHarm severity: {}".format(response.self_harm_result.severity)
if response.violence_result:
    text = text + "\n\tViolence severity: {}".format(response.violence_result.severity)
if response.sexual_result:
    text = text + "\n\tSexual severity: {}".format(response.sexual_result.severity)

print(text)

Content Safety Results:
	Hate severity: 0
	SelfHarm severity: 0
	Violence severity: 0
	Sexual severity: 0


#### Read lines from a file and check for content safety

In [4]:
from azure.core.exceptions import HttpResponseError
from azure.ai.contentsafety.models import AnalyzeTextOptions, TextCategory
import os

from azure_content_safety_setup import get_content_safety

text_file = 'safe_content.txt'
text_path = os.path.join(os.getcwd(),'./data/', text_file)

# Read file line by line and check for content safety
i = 0
with open(text_path) as f:
    for line in f:
        print("[%s]:[%s]:%s" %(text_file, i, line))
        # calling my function to do the job
        result = get_content_safety(client=client, text_input=line)
        print(result)
        i = i + 1

[safe_content.txt]:[0]:You are awesome.

Content Safety Results:
	Hate severity: 0
	SelfHarm severity: 0
	Violence severity: 0
	Sexual severity: 0
[safe_content.txt]:[1]:The world is great.

Content Safety Results:
	Hate severity: 0
	SelfHarm severity: 0
	Violence severity: 0
	Sexual severity: 0


#### Add a block list

In [5]:
from azure_content_safety_setup import add_or_update_block_list

MY_BLOCK_LIST = 'TR_Custom_Blocked_Text_List'
MY_BLOCK_LIST_DESCRIPTION = 'TRs fun list of blocked text strings for testing and demo of the Azure Content Safety APIs'
added_block_list = add_or_update_block_list(client = client, 
                                            block_list_name = MY_BLOCK_LIST, 
                                            block_list_description = MY_BLOCK_LIST_DESCRIPTION)
print("Created Block list name: %s" %(added_block_list))


Blocklist created or updated: 
Created Block list name: TR_Custom_Blocked_Text_List


#### Add blocked text strings

In [6]:
from azure_content_safety_setup import add_blocked_items

my_blocked_text_array = ["kil'ya", "f**king", "m0**ns"]
added_text_list = add_blocked_items(
                        client = client, 
                        block_list_name = MY_BLOCK_LIST, 
                        block_text_array = my_blocked_text_array
                    )
print("Blocked Text items: %s" %(added_text_list))


Add block items failed: 
Error code: InvalidRequestBody
Error message: Some items already exist in list TR_Custom_Blocked_Text_List. items: [m0**ns,kil'ya,f**king]. | Request Id: a60c60b9-97bc-48be-9ca0-811bf5cad803, Timestamp: 2023-06-14T19:33:09Z.
(InvalidRequestBody) Some items already exist in list TR_Custom_Blocked_Text_List. items: [m0**ns,kil'ya,f**king]. | Request Id: a60c60b9-97bc-48be-9ca0-811bf5cad803, Timestamp: 2023-06-14T19:33:09Z.
Code: InvalidRequestBody
Message: Some items already exist in list TR_Custom_Blocked_Text_List. items: [m0**ns,kil'ya,f**king]. | Request Id: a60c60b9-97bc-48be-9ca0-811bf5cad803, Timestamp: 2023-06-14T19:33:09Z.
Blocked Text items: FAILED


#### Analyze text with block list

In [7]:
from azure_content_safety_setup import get_content_safety_custom

result = get_content_safety_custom(client = client, block_list_names = [MY_BLOCK_LIST], text_input = "I will kil'ya")
print(result)

Content Safety Results (blank if none found):
  Blocklist match results: 
	Block item hit in text, Offset=7, Length=6
	BlocklistName: TR_Custom_Blocked_Text_List, BlockItemId: 1f58ef20-0968-4de6-9231-37d5cedf532e
	BlockItemText: kil'ya


#### Analyze image

In [8]:
from azure_content_safety_setup import get_image_safety
import os

image_file = 'image.jpg'
image_path = os.path.join(os.getcwd(),'./data/', image_file)

result = get_image_safety(client = client, image_filepath = image_path)
print(result)

Image Content Safety Results in file C:\Users\tibarar\OneDrive - Microsoft\Desktop\MyProjects\open-ai\ResponsibleAI\./data/image.jpg:
	Hate severity: 0
	SelfHarm severity: 0
	Violence severity: 2
	Sexual severity: 0
